Notebook to evaluate detection from saved model

In [ ]:
import os
import glob
import tensorflow as tf
import PIL.Image as Image
import matplotlib.pyplot as plt
import numpy as np
import utils.visualization_utils as viz_utils
from utils import np_box_list
from utils import np_box_list_ops
from data_decoders import tf_example_decoder

%matplotlib inline

In [ ]:
# load model saved with export_model.sh
tf.keras.backend.clear_session()
model_type = 'faster_rcnn'
detect_fn = tf.saved_model.load('/home/tensorflow/work/models/' + model_type + '_resnet50/export/saved_model')

In [ ]:
# check model performance on evaluation set
tfrecord_path = 'data/eval.record'
dataset = tf.data.TFRecordDataset(tfrecord_path)
decoder = tf_example_decoder.TfExampleDecoder()

# initialize iou/class score lists for calculating statistics
iou_all = []
class_score_all = []

# for plotting a few examples...
category_index = {
    1: {'id': 1, 'name': 'truck'}
}

# iterate over evaluation images, perform detection, save class scores and ious
# plot the first few with detected boxes
for idx, example in enumerate(dataset.as_numpy_iterator()):
    im_dict = decoder.decode(example)
    truth_boxes = np_box_list.BoxList(im_dict['groundtruth_boxes'].numpy())
    detections = detect_fn(tf.expand_dims(im_dict['image'], axis=0))
    detect_boxes = np_box_list.BoxList(detections['detection_boxes'].numpy()[0])

    class_score_all.append(np.squeeze(detections['detection_multiclass_scores'].numpy()))
    iou_all.append(np_box_list_ops.iou(truth_boxes, detect_boxes)[0])

    if idx < 5:
        im_detect = im_dict['image'].numpy()

        viz_utils.visualize_boxes_and_labels_on_image_array(im_detect,
            detections['detection_boxes'][0].numpy(), 
            detections['detection_classes'][0].numpy().astype(np.int32), 
            detections['detection_scores'][0].numpy(),
            category_index,
            use_normalized_coordinates=True,
            min_score_thresh=0.75)

        plt.figure()
        plt.imshow(im_detect)
        plt.show()

In [ ]:
n_tgts = len(iou_all) # total number of true targets (1 per image)
n_detections = np.shape(iou_all[0])[0] # number of detections per image
tp = 0 # true positives
fp = 0 # false positives
fn = 0 # false negatives

# define thresholds for positive detection
iou_thresh = 0.5
class_thresh = 0.75

# iterate over images
for iou_im, class_score_im in zip(iou_all, class_score_all):
    # initialize per image true/false positives (so i can check for false negative, ie no detection)
    tp_im = 0
    fp_im = 0
    # iterate over detections for this image
    for iou, class_score in zip(iou_im, class_score_im): # gotta name these something clearer
        if (iou > iou_thresh) and (class_score[1] > class_thresh):
            tp_im += 1
        elif (iou < iou_thresh) and (class_score[1] > class_thresh):
            fp_im += 1

    if tp_im > 0:
        tp += 1
    else:
        fn += 1

    fp += fp_im

p_dc = tp / n_tgts # probability of correct detection/classification
p_fa = fp / n_tgts # probability of false alarm

print(f'Probability of correct detection/classification = {p_dc}')
print(f'Probability of false alarm = {p_fa}')